In [1]:
from param import Param
from sde import *
from fparam import *
import numpy as np
import math
from matplotlib import pyplot as plt

In [2]:
# First we calibrate using the heston model
params = {
    "h_0"  : Param(.11/math.sqrt(21)),
    "theta": Param(.17/math.sqrt(252)),
    "rho"  : Param(0.4),
    "kappa": Param(.2),
    "xi"   : Param(1.2),
    "dt"   : Param(1/252),
    "r"    : Param(0.0411/252),
    "S_0"  : Param(100)
}

# Model functions
def feller(params):
    k     = params["kappa"].eval()
    theta = params["theta"].eval()
    xi    = params["xi"].eval()
    return 2 * k * theta > xi * xi

def mu_h(h_t, t, params=None):
    k = params["kappa"].eval()
    theta = params["theta"].eval()
    return k * (theta - h_t)

def v_h(h_t, t, params=None):
    xi = params["xi"].eval()
    return xi * math.sqrt(max(h_t, 0.))

def mu(S_t, t, params=None):
    r = params["r"].eval()
    return r * S_t

def v(S_t, t, cir, params=None):
    vol = cir.nodes[t-1].eval()
    return math.sqrt(max(vol, 0.)) * S_t

drift_h  = FParam(mu_h, params)
vol_h    = FParam(v_h, params)
T   = 252
N   = 500

cir_sim = EulerSimulation(drift_h, vol_h, params["h_0"], T, params["dt"], N)

drift_s = FParam(mu, params)
vol_s   = FParam(v, params)

simulation = StochVolSimulation(drift_s, vol_s, params["S_0"], cir_sim, T, params["dt"], N, rho=0.4)

In [3]:
targets = {
    1: {95: 6.5757, 100: 2.8223, 105: 0.6335},
    2: {95: 8.1165, 100: 4.3850, 105: 1.7263},
    3: {100: 6.0865, 105: 3.1820, 110: 1.2347},
    4: {100: 7.7710, 105: 4.7369, 110: 2.4165}
}

def plot_sims(cir_sim, simulation):
    for i in range(N):
        plt.plot(cir_sim.results[:, i])
    plt.xlabel("Timesteps (252 total)")
    plt.ylabel("Daily Volatility")
    plt.title("Volatility Simulation")
    plt.show()

    for i in range(N):
        plt.plot(simulation.results[:, i])
    plt.xlabel("Timesteps (252 total)")
    plt.ylabel("Index Price")
    plt.title("Index Price Simulation")
    plt.show()
    
def call_price(result, targets, r):
    qt = T // 4
    out = dict()
    sse = 0.
    for m in targets:
        out[m] = dict()
        for strike in targets[m]:
            out[m][strike] = np.maximum((result[(qt*m), :] - strike), 0).mean() * np.exp(- r * qt * m)
            sse += (out[m][strike] - targets[m][strike])**2
    return out, sse

def run_sims(cir_sim, simulation):
    cir_sim.run_all()
    simulation.run_all()

In [ ]:
# Calibration
step = 5
theta_grid = np.linspace(0.02, 0.03, step)
kappa_grid = np.linspace(10, 20, step)
xi_grid    = np.linspace(0.00001, 0.0001, step)
opt_comb   = None
opt_sse    = -1.
opt_out    = None
ct = 0
    
for theta in theta_grid:
    for kappa in kappa_grid:
        ct += 1
        for xi in xi_grid:
            params["theta"].val = theta
            params["kappa"].val = kappa
            params["xi"].val    = xi
            if feller(params):
                run_sims(cir_sim, simulation)
                out, sse = call_price(simulation.results, targets, params["r"].eval())
                if opt_sse < 0. or sse < opt_sse:
                    opt_sse = sse
                    opt_comb = (theta, kappa, xi)
                    opt_out = out
    if ct % step == 0:
        print(f"Calibration {ct // step}/{step} complete.")

Calibration 1/10 complete.
Calibration 2/10 complete.
Calibration 3/10 complete.
Calibration 4/10 complete.
Calibration 5/10 complete.
Calibration 6/10 complete.


In [ ]:
print("SSE: = ", sse)
print("Optimal Combination = ", opt_comb)
print("Call Prices = ", opt_out)